# Coola-Coola

## Introduction to optimization and operations research.

Michel Bierlaire


The objective of this first set of exercises is to demonstrate how to use an optimization algorithm in Python.
Rather than treating such algorithms as "black boxes," it is important to critically evaluate the solutions
they produce. You should assess whether the results make sense, and if not, investigate the cause and refine the
model or approach accordingly.

The company Coola-Coola Ltd. wants to design a can of soda of volume
0.33 liters. They need to set the dimensions (in centimeters) of this
can to use the minimum amount of aluminium, knowing that the form of the can is a
perfect cylinder, and the thickness of the aluminium is the same
everywhere. Write the problem as an optimization problem. Then, write a Python code to solve it, using the
[scipy.minimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) function.

Make sure to read the documentation first:
[click here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html)

# Formulation of the optimization problem

Before proceeding with the implementation, formulate the problem as a constrained optimization problem, where the
constraint is written as $$g(x)=0.$$ No need to implement anything yet.

## What are the decision variables?

There are two decision variables: the radius ($r$) and the height ($h$) of the cylinder.

## What is the objective function?

The objective is to minimize the surface area of the cylinder. It is composed of
- the side of the cylinder, that is a rectangle of width $2 \pi r$ and height $h$: $2 \pi rh$,
- the top of the cylinder, that is a circle or radius $r$: $\pi r^2$,
- the bottom of the cylinder, that has the same surface as the top: $\pi r^2$.
Therefore, the total surface is $$2 \pi r (h + r)$$.

The constraint imposes that the volume of the cylinder must be 330 cm³. The volume of the can is $\pi r²h$, so that
the constraint is $\pi r²h = 330$. In order to write it as $g(x)=0$, we define $$g(r, h) = \pi r²h - 330.$$

Finally, do not forget to impose the decision variables to be non negative: $r \geq 0$, $h \geq 0$.

Therefore, the optimization problem is written
$$ \min_{r, h} 2 \pi r (h + r)$$
subject to
$$ \pi r²h - 330 = 0,$$
$$ r \geq 0,$$
$$ h \geq 0.$$

# Calculating the solution

Now, we start the Python implementation to solve the optimization problem using `scipy.optimize.minimize`.

Refer to the documentation for the minimize function:
https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html

Import necessary packages.

In [ ]:
import numpy as np
from scipy.optimize import minimize



We first define a function calculating the surface.

In [ ]:
def surface(radius: float, height: float) -> float:
    return 2 * np.pi * radius * (radius + height)



We then define a function calculating the volume, that is $\pi r^2 h$.

In [ ]:
def volume(radius: float, height: float) -> float:
    return np.pi * radius * radius * height



Prepare the input for the optimization algorithm.

In [ ]:
def objective_function(x: list[float]) -> float:
    """point[0]: radius, point[1]: height"""
    return surface(radius=x[0], height=x[1])


def constraint(x: list[float]) -> float:
    """x[0]: radius, x[1]: height
    volume = 330
    """
    the_volume = 330
    return volume(radius=x[0], height=x[1]) - the_volume



Lower bounds: 0. No upper bounds.

In [ ]:
bounds = [(0, None), (0, None)]



## Starting point.

The choice of a starting point is a key aspect in optimization. Selecting a good one is often not straightforward,
and different algorithms treat starting points in different ways. If the initial guess is infeasible, many solvers
will attempt to adjust it to a feasible point, but success is not always guaranteed. In addition, the quality and
even the location of the final solution may strongly depend on the chosen starting point. In this and the following
exercises, we encourage you to try out different initial guesses and observe how they influence the results.

In [ ]:

x0 = [1, 1]


Run the algorithm. Check the documentation of the minimize function of `scipy`.

In [ ]:
optimization_result = minimize(
    fun=objective_function,
    x0=np.array(x0),
    constraints={'type': 'eq', 'fun': constraint},
    bounds=bounds,
)



Print the raw results. Note that 'success' is set to True.

In [ ]:
print(optimization_result)


Print the solution with 3 significant digits. We expect:

- Optimal radius: 3.74 cm.
- Optimal height: 7.49 cm.
- Surface: 264 cm2.

In [ ]:
print(f'Optimal radius: {optimization_result.x[0]:.3g} cm.')
print(f'Optimal height: {optimization_result.x[1]:.3g} cm.')
print(f'Surface: {optimization_result.fun:.3g} cm2.')


At first sight, solving an optimization exercise may look straightforward,
but in practice it can be trickier than expected.

- Depending on the starting point, the algorithm may converge to a local
minimum instead of the global one.
- Numerical issues due to finite precision arithmetic on computers can
affect convergence and accuracy.
- If the starting point is infeasible (i.e., it violates some constraints),
certain algorithms may not be able to find a feasible solution at all.
- Importantly, it is generally impossible to anticipate the outcome
(success, failure, or type of solution found) based only on the chosen
starting point.

To better understand these challenges, try running the same code with
different starting points and compare the results.

In [ ]:
x0 = [0, 0]
x0 = [2, 2]
x0 = [3, 3]
x0 = [3, 2]